# LiCE minimal example

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from LiCE.data.DataHandler import DataHandler
from LiCE.data.Features import Monotonicity
from LiCE.spn.SPN import SPN
from LiCE.lice.LiCE import LiCE

from nn_model import NNModel

## Prepare data and models
### Setup data context (mutability, feature types...)

In [2]:
data = pd.read_csv("data/GMSC/cs-training.csv", index_col=0).dropna()
X = data[data.columns[1:]]
y = data[["SeriousDlqin2yrs"]]

# set bounds on feature values
# either fixed by domain knowledge
bounds = {"RevolvingUtilizationOfUnsecuredLines": (0, 1), "DebtRatio": (0, 2)}
# or take them from the data
for col in X.columns:
    if col not in bounds:
        bounds[col] = (X[col].min(), X[col].max())

config = {
    "categ_map": {}, # categorical features, map from feature names to a list of categ values, e.g. {"sex": ["male", "female"] | if one provides an empty list with a feature name, then all unique values are taken as categories - note that training split does not have to include all values...
    "ordered": [], # list of featurenames that contain ordered categorical values, e.g. education levels
    "discrete": [
        "NumberOfTime30-59DaysPastDueNotWorse",
        "age",
        "NumberOfOpenCreditLinesAndLoans",
        "NumberOfTimes90DaysLate",
        "NumberRealEstateLoansOrLines",
        "NumberOfTime60-89DaysPastDueNotWorse",
        "NumberOfDependents",
    ], # contiguous discrete fearures
    "bounds_map": bounds, # bounds on all contiguous values
    "immutable": ["NumberOfDependents"], # features that cannot change
    "monotonicity": {"age": Monotonicity.INCREASING}, # domain constrain on whether
    "causal_inc": [], # causal increase, pairs of features where if one increases, the other one must increase as well, e.g. [("education", "age")]
    "greater_than": [], # inter-feature dependence, one feature must always be > anohter feature, a list of pairs, e.g. [("# total missed payments",  "# missed payments last year")]
}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
dhandler = DataHandler(
    X_train,
    y_train,
    **config,
    # optionally, one can provide the list of feature names (and target name) but here we pass pandas dataframe (and a series) with named columns, which will be taken as feature names
)

X_train_enc = dhandler.encode(X_train, normalize=True, one_hot=True)
y_train_enc = dhandler.encode_y(y_train, one_hot=False)

### Train a Neural Network

here we utilize our wrapper, but any ReLU NN that can be exported to ONNX should work

In [ ]:
nn = NNModel(dhandler.encoding_width(True), [20, 10], 1)
nn.train(X_train_enc, y_train_enc)
# output it to ONNX file
nn.save_onnx("tmp_nn.onnx")

### Train an SPN

In [ ]:
# this can take long... 
# setting "min_instances_slice":1000 argument leads to faster training by allowing leaves to be formed on >=1000 samples (default is 200)

# data should be numpy array of original (non-encoded) values, should include the target as last feature
spn_data = np.concatenate([X_train.values, y_train.values], axis=1) 
spn = SPN(spn_data, dhandler, normalize_data=False, learn_mspn_kwargs={"min_instances_slice":1000})

## Find Counterfactual Explanations

In [11]:
top_n = 1 # top-n counterfactuals to look for (obtaining more than 1 is implemented only for Gurobi solver)
time_limit = 120 # number of seconds to look for a counterfactual
# solver to choose
solver_name = "gurobi" # Gurobi solver was used in the original experiments. 
# solver_name = "appsi_highs" # This is an open-source solver. Install with: $ pip install highspy

lice = LiCE(
    spn,
    nn_path="tmp_nn.onnx",
    data_handler=dhandler,
)

# take a sample
# np.random.seed(42)
np.random.seed(1)
i = np.random.randint(X_test.shape[0])
print(i, X_test.shape[0])

sample = X_test.iloc[i]
enc_sample = dhandler.encode(X_test.iloc[[i]]) # to keep 2 dimensions
# enc_sample = dhandler.encode(X_test.iloc[i]) # to keep 2 dimensions
prediction = nn.predict(enc_sample) > 0 # we assume binary classification without sigmoid activation in the end
prediction

235 24054


array([[False]])

### Thresholding variant

In [12]:
lls = spn.compute_ll(spn_data)
quartile_ll = np.quantile(lls, 0.25) # threhsold on CE likelihood

thresholded = lice.generate_counterfactual(
    sample,
    not prediction,
    ll_threshold=quartile_ll,
    n_counterfactuals=top_n, 
    time_limit=time_limit, 
    solver_name=solver_name
)
print(thresholded)
lice.stats

[array([0.994575, 63.0, 1.0, 0.004011, 3995.62069, 4.0, 2.0, 0.0, 1.0, 0.0],
      dtype=object)]


{'time_total': 11.4788535819971,
 'time_solving': 9.91448436799692,
 'time_building': 1.5409484319970943,
 'optimal': True,
 'll_computed': [-21.250178059015703],
 'dist_computed': [2.6760164239499087]}

### Optimizing variant

In [13]:
alpha = 0.1 # coefficient of the linear combination in the objective
# taking a value approximately equal to (mean CE distance)/(mean mean CE log-likelihood) works well.
# running the thresholding variant first and then estimating some alpha is a reasonable approach

optimized = lice.generate_counterfactual(
    sample,
    not prediction,
    ll_opt_coefficient=alpha,
    n_counterfactuals=top_n, 
    time_limit=time_limit, 
    solver_name=solver_name
)
print(optimized)
lice.stats

[array([0.762035, 63.0, 1.0, 0.004011, 3528.198147, 4.0, 2.0, 0.0, 1.0,
       0.0], dtype=object)]


{'time_total': 7.681302868004423,
 'time_solving': 6.24439866701141,
 'time_building': 1.4052089839969995,
 'optimal': True,
 'll_computed': [-25.047672688979794],
 'dist_computed': [1.1741684347788122]}